In [ ]:
import numpy as np
import os
import pandas as pd
import h5py
import seaborn as sns
import matplotlib.pyplot as plt
from toolbox.utils import set_random_seed
from toolbox.plt_utils import density_scatter

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

%load_ext autoreload
%autoreload 2

In [3]:
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
# for TF To find gpu
os.environ["LD_LIBRARY_PATH"] = f"{os.environ['CONDA_PREFIX']}/lib:" + os.environ.get("LD_LIBRARY_PATH", "")

In [4]:
# set random seed for reproducibility
set_random_seed(seed=42)

2025-12-16 09:12:39.284298: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/valehvpa/miniconda3/envs/hist6/lib:/usr/local/cuda/lib64:/usr/local/cuda/lib64
2025-12-16 09:12:39.284320: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


TensorFlow not installed, skipping tf seeding.


/home/valehvpa/miniconda3/envs/hist6/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Seed set to 42


# chrombpoent debugging

In [6]:
p = "/home/valehvpa/projects/scCisTrans/chrombpnet-pytorch/output/fold_0/metrics.csv"
df = pd.read_csv(p)
df


,epoch,step,train_count_loss,train_count_pearson,train_loss,train_profile_loss,train_profile_pearson,val_count_loss,val_count_pearson,val_loss,val_profile_loss,val_profile_pearson
0,0,3190,NaN,NaN,NaN,NaN,NaN,0.909426,0.617945,1060.226562,991.110596,0.756252
1,0,3190,1.341471,0.411049,1157.842896,1055.890747,0.734339,NaN,NaN,NaN,NaN,NaN
2,1,6381,NaN,NaN,NaN,NaN,NaN,0.877033,0.650909,1016.791260,950.136780,0.775693
3,1,6381,0.960519,0.612061,1038.062744,965.063049,0.769962,NaN,NaN,NaN,NaN,NaN
4,2,9572,NaN,NaN,NaN,NaN,NaN,0.773535,0.671603,1002.859375,944.070923,0.778694
5,2,9572,0.899259,0.644059,998.754761,930.410339,0.780884,NaN,NaN,NaN,NaN,NaN
6,3,12763,NaN,NaN,NaN,NaN,NaN,0.733697,0.690722,991.277527,935.517151,0.783298
7,3,12763,0.849934,0.669321,975.065735,910.470398,0.786952,NaN,NaN,NaN,NaN,NaN
8,4,15954,NaN,NaN,NaN,NaN,NaN,0.836810,0.696796,995.524963,931.927734,0.785081
9,4,15954,0.815451,0.687572,958.758911,896.784119,0.790860,NaN,NaN,NaN,NaN,NaN


In [10]:
import wandb
wandb.init(project="histobpnet", name="manual_lei_chrombpnet_metrics_log2")
# artifact = wandb.Artifact("metrics_csv", type="dataset")
# artifact.add_file(p)
# wandb.log_artifact(artifact)
table = wandb.Table(dataframe=df)
wandb.log({"metrics_table": table})

## subsmaple

In [14]:
p = "/large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_chrombpnet_tuto/lei_peaks.bed"
# read from bed and sumsample
lei_peaks_df = pd.read_csv(p, sep="\t", header=None)

p = "/large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_chrombpnet_tuto/negatives/lei_negatives.bed"
lei_negatives_df = pd.read_csv(p, sep="\t", header=None)

print(lei_peaks_df, lei_negatives_df)

           0          1          2            3     4  5        6          7  \
0       chr1  100027916  100029541  Peak_168774   375  .  3.46597   37.52287   
1       chr1  100027916  100029541  Peak_212397   247  .  2.93255   24.75966   
2       chr1  100027916  100029541  Peak_231027   212  .  2.75187   21.27480   
3       chr1  100027916  100029541  Peak_240800   198  .  2.67162   19.80458   
4       chr1  100027916  100029541   Peak_64424  1000  .  8.38628  201.28123   
...      ...        ...        ...          ...   ... ..      ...        ...   
269713  chrY   11103101   11103309  Peak_137563   548  .  4.84445   54.86904   
269714  chrY   12394949   12395289  Peak_229097   217  .  3.12339   21.72005   
269715  chrY   12394949   12395289  Peak_261400   172  .  2.83655   17.28688   
269716  chrY   18992804   18993052  Peak_111927   802  .  5.92807   80.24894   
269717  chrY    4030688    4031055  Peak_100875   964  .  6.56550   96.46984   

                8     9  
0        35.5

In [15]:
!wc -l /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_chrombpnet_tuto/lei_peaks.bed
!wc -l /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_chrombpnet_tuto/negatives/lei_negatives.bed

/bin/bash: /home/valehvpa/miniconda3/envs/hist6/lib/libtinfo.so.6: no version information available (required by /bin/bash)
269718 /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_chrombpnet_tuto/lei_peaks.bed
/bin/bash: /home/valehvpa/miniconda3/envs/hist6/lib/libtinfo.so.6: no version information available (required by /bin/bash)
472962 /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_chrombpnet_tuto/negatives/lei_negatives.bed


In [23]:
# randomly subsample 1000 peaks and negatives and save to new bed files
n_sub = 40_000
lei_peaks_df_sampled = lei_peaks_df.sample(n=n_sub, random_state=42)
lei_peaks_df_sampled.to_csv(
    f"/large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_chrombpnet_tuto/lei_peaks_ss_{n_sub}.bed",
    sep="\t", header=False, index=False)

lei_negatives_df_sampled = lei_negatives_df.sample(n=n_sub, random_state=42)
lei_negatives_df_sampled.to_csv(
    f"/large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_chrombpnet_tuto/negatives/lei_negatives_ss_{n_sub}.bed",
    sep="\t", header=False, index=False)

In [22]:
!head /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_chrombpnet_tuto/lei_peaks_ss_1000.bed
!wc -l /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_chrombpnet_tuto/lei_peaks_ss_1000.bed

!head /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_chrombpnet_tuto/negatives/lei_negatives_ss_1000.bed
!wc -l /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_chrombpnet_tuto/negatives/lei_negatives_ss_1000.bed

/bin/bash: /home/valehvpa/miniconda3/envs/hist6/lib/libtinfo.so.6: no version information available (required by /bin/bash)
chr12	6532566	6533036	Peak_42711	1000	.	4.30188	349.82059	347.21072	139
chr5	135226186	135226478	Peak_87987	1000	.	4.99816	122.44863	120.19123	135
chr2	9070971	9071448	Peak_60357	1000	.	9.03456	221.08578	218.64418	296
chr3	38003407	38004705	Peak_181021	329	.	1.86632	32.93175	31.00591	917
chr1	226449800	226450190	Peak_113019	787	.	5.3749	78.77083	76.63297	228
chr1	120267928	120268479	Peak_79603	1000	.	4.76701	144.65266	142.34657	139
chr7	139767293	139768409	Peak_184825	316	.	3.05146	31.68775	29.77085	578
chr4	159118033	159118348	Peak_48318	1000	.	8.39765	299.57346	297.02344	132
chr9	14508888	14510011	Peak_267883	164	.	2.19029	16.47649	14.71437	461
chr1	151520149	151520692	Peak_212136	248	.	3.28844	24.82766	22.96885	352
/bin/bash: /home/valehvpa/miniconda3/envs/hist6/lib/libtinfo.so.6: no version information available (required by /bin/bash)
1000 /large_storage/good

Commands to run to "compare" the two models:

```bash
ca hist6

python /home/valehvpa/projects/scCisTrans/histobpnet/scripts/train/main.py \
    --command train \
    --name chrombpnet_train_k562_lei_compare_seed_ev \
    --model_type chrombpnet \
    --output_dir /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_chrombpnet_tuto/training \
    --fasta /large_storage/goodarzilab/valehvpa/refs/hg38/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta \
    --chrom_sizes /large_storage/goodarzilab/valehvpa/refs/hg38/hg38.chrom.sizes \
    --peaks /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_chrombpnet_tuto/lei_peaks_ss_40000.bed \
    --negatives /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_chrombpnet_tuto/negatives/lei_negatives_ss_40000.bed \
    --bigwig /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_chrombpnet_tuto/lei_unstranded.bw \
    --bias_scaled /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_chrombpnet_tuto/pretrained/ENCSR467RSV/fold_0/model.bias_scaled.fold_0.ENCSR868FGK.h5 \
    --adjust_bias \
    --fold 0 \
    --gpu 0 \
    --cvd 1 \
    --num_workers 8
```

```bash
ca chrombpnetpytorch

chrombpnet train --peaks /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_chrombpnet_tuto/lei_peaks_ss_40000.bed --negatives /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_chrombpnet_tuto/negatives/lei_negatives_ss_40000.bed --bigwig /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_chrombpnet_tuto/lei_unstranded.bw --bias /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_chrombpnet_tuto/pretrained/ENCSR467RSV/fold_0/model.bias_scaled.fold_0.ENCSR868FGK.h5 --fasta /large_storage/goodarzilab/valehvpa/refs/hg38/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta --adjust_bias --num_workers 8
```